# Statistical Arbitrage: Asset clustering, market-exposure minimization, and high-frequency explorations

Two strategies are indtroduced in this paper. 
* mean reverting residual of a cluster of assets whose weights are selected so as to minimize market exposure.
* a portfolio of pairs, weighted proportional to a measure of mean reversion speed. 

## 1. Introduction

Statistical arbitrage is the systematic exploition of perceived mispricings of similar assets. 
Three pillars are needed: 
1. a measure of similarity of assets
2. a measure of pricing mismatch
3. a confidence metric for mismatch

## 2. Model Basics - Signal Generation

### 2.a Set ups

For two correlated assets, we can represent the relationship between their returns as: 
$$ \frac{dQ}{Q} = \alpha + \beta\frac{dP}{P} + \epsilon $$
where $\frac{dQ}{Q}$ and $\frac{dP}{P}$ are correlated. 
* $\alpha$ is usually small, authors take it as zero
* set $dX_t$ as the mismatch $\epsilon$. 
$$dX_t = \frac{dQ}{Q} - \beta\frac{dP}{P}$$
__$dX_t$ usually represents a two asset portfolio while it can also model the residual of a general cluster of assets.__ 

### 2.b Assumption

The assumption of the statistical arbitrage is that the mismatch is mean reverting, i.e. $dX_t$ follows an __Ornstein-Uhlenbeck Process__.
$$dX_t = \kappa\left(m - X_t\right)dt + \sigma dW_t$$ 
where $\kappa > 0$.  
$\kappa$ is the mean reversion speed, $m$ is the equilibrium mean, $\sigma$ is the volatility. 

### 2.c Solution of OU process & Signal Generation

#### Derivation

Ornstein-Uhlenbeck process  
$$dX_t = \kappa\left(m - X_t\right) + \sigma dW_t$$  
Multiply it with $e^{\kappa t}$
$$dX_te^{\kappa t} = \kappa e^{\kappa t}X_tdt + e^{\kappa t}dX_t = e^{\kappa t}\kappa mdt + e^{\kappa t}\sigma dW_t$$

Taking integration
$$\int_{t_0}^{t_0 + \Delta t}dX_se^{\kappa s} = \int_{t_0}^{t_0 + \Delta t}e^{\kappa s}\kappa mds + \sigma\int_{t_0}^{t_0 + \Delta t}e^{\kappa s} dW_s$$
$$X_{\left(t_0 + \Delta t\right)}e^{\kappa \left(t_0 + \Delta t\right)} - X_{t_0}e^{\kappa t_0} = m \cdot\left(e^{\kappa \left(t_0 + \Delta t\right)} - e^{\kappa t_0}\right) + \sigma\int_{t_0}^{t_0 + \Delta t}e^{\kappa s} dW_s $$

Solution
$$X_{\left(t_0 + \Delta t\right)} = e^{-\kappa \Delta t}X_{t_0} + m\cdot\left(1 - e^{-\kappa \Delta t}\right) + \sigma\int_{t_0}^{t_0 + \Delta t}e^{\kappa \left(s - \left(t_0 + \Delta t\right)\right)} dW_s$$

Variance of the random part
$$Var\left(\sigma\int_{t_0}^{t_0 + \Delta t}e^{\kappa \left(s - \left(t_0 + \Delta t\right)\right)} dW_s\right) = \sigma^2\int_{-\Delta t}^{0} e^{2\kappa s}ds = \sigma^2 \frac{1 - e^{-2\kappa \Delta t}}{2\kappa}$$

#### Discretized Version

$$ X_{n + 1} = a + bX_n + \epsilon_{n + 1}$$ 
where $a = m\left(1 + e^{-\kappa \Delta t}\right), b = e^{-\kappa \Delta t}, Var\left(\epsilon\right) = \sigma^2\frac{1 - e^{-2\kappa\Delta t}}{2\kappa}$.  
And $\Delta t$ is the time difference between the two timestamps.  
__With the discretized version, we can estimate $\kappa, m, \sigma%=$.__

#### Equilibrium & Signal Derivation

Let $\Delta t \rightarrow \infty$, we have $$X_t = m + \epsilon$$ where $\epsilon \sim N\left(0, \frac{\sigma ^2}{2\kappa}\right)$.   
Since X follows a normal distribution, we can treat the standardized X as the signal. 
$$ s = \frac{\sqrt{2\kappa}\left(X - m\right)}{\sigma}$$
A negative signal means the performance of portfolio is underpriced (long 1 Q and short $\beta$ P with negative excess return), and good time for long positions.  
A positive signal means the performance of portfolio is overpriced, and good time for short. 

From the paper [Statistical Arbitrage in the US Equities Market](https://www.math.nyu.edu/faculty/avellane/AvellanedaLeeStatArb071108.pdf), the authors inherit the conclusion on thresholds of the signals. 
$s_{long,enter} = -1.25, s_{long,exit} = -0.5, s_{short,enter} = 1.25, s_{short,exit} = 0.75$

![Figure 1](Figure1.png)

## 3. Strategy 1: Cluster Trading

A statistical arbitrage strategy - minimizing exposure to market factors.   
Using PCA to reduce dimensions of the asset universe to three bases. And use the three bases to represent all the universe. 

### 3.1 Asset Selection - Subtractive Clustering in Orthogonal Eigenspace

* Universe: M assets, T days, a bank excess return matrix of T * M, $R_{T \times M}$  
* Normalize the excess returns (substracting bank return) by subtracting the mean and dividing by the std, $Y_{T \times M}$. 
* Use singular value decomposition for Y, we have: 
$$Y_{T\times M} \approx U_{T\times K}\cdot\Sigma_{K\times K}\cdot V_{M\times K}^T = F_{T\times K}\cdot B_{K\times M}, $$ where $\Sigma$ is the singular value diagnal matrix ($\sigma_i^{svd}, i=1,2,3,...,K$) and K is the number of bases from PCA. 

__Columns of $F$ are bases and orthonomal. The columns of $B$ are like the coefficients of the bases.__

### 3.1a Subtractive Clustering Algo

Subtractive clustering algorithm is used for clustering. [Introduction of Subtractive Clustering Algo](https://scialert.net/fulltext/?doi=itj.2008.356.360)  

Sudo Code: 
* Calculate the potential of each data points  
$$P_i = \sum_{j=1}^Mexp\left(-\sum_{t = 1}^K\frac{\left(\beta_{it} - \beta_{jt}\right)^2}{\left(r_t/2\right)^2}\right), $$ where $$r_t = \frac{\left(1/\sigma_t^{svd}\right)^2}{\sum_{s=1}^K\left(1/\sigma_s^{svd}\right)^2}$$
* Select the point with highest potential as the first cluster center, say i = c1. 
$$P_i = P_i - P_{c1} \cdot \sum_{j=1}^Mexp\left(-\sum_{t = 1}^K\frac{\left(\beta_{it} - \beta_{c1,t}\right)^2}{\left(1.5 * r_t/2\right)^2}\right)$$
* Select the point with highest potential as the next cluster, say c2 and updates the potential as above, just replacing c1 by c2, and by ck for later rounds. 
* End as the stopping criterion is met. 

### 3.2 Clustering Signal Generation

We want to keep the exposure to market risk at minimum. 
In pair trading, we use a formula like 
$$ dX_t = dr_P - \beta dr_Q$$
and $dX_t$ is the return of a portfolio containing assets P & Q.  

While for trading a set of securirities, say P securities, we have a formula like: 
$$ dX_t = Y_{T \times P}\overrightarrow{w}$$ where $\overrightarrow{w}$ is the weight vector. 

To minimize the risk exposure, the target is to $min\ \overrightarrow{w}^T\mathbb{Cov}\left(Y\right)\overrightarrow{w}$ s.t. $\overrightarrow{w}\cdot\mathbb{1} = 1$, plus the non negative (non positive) return for long (short) position. 


$$\overrightarrow{w}^T\mathbb{Cov}\left(Y_{T\times P}\right)\overrightarrow{w}$$ 
$$= \overrightarrow{w}^T\mathbb{Cov}\left(F_{T\times K}\cdot B_{K\times P}\right)\overrightarrow{w}$$
$$ = \left(F_{T\times K}B_{K\times P}\overrightarrow{w}\right)^T \cdot \left(F_{T\times K}B_{K\times P}\overrightarrow{w}\right)$$
$$ = \left(B_{K\times P}\overrightarrow{w}\right)^T\cdot F^T\cdot F\cdot\left(B_{K\times P}\overrightarrow{w}\right)$$
$$ =  \left(B_{K\times P}\overrightarrow{w}\right)^T\left(B_{K\times P}\overrightarrow{w}\right)$$
$$ = ||B_{K\times P}\overrightarrow{w}|| ^ 2$$
F is orthonormal matrix. 

$$min \ \overrightarrow{w}^T\mathbb{Cov}\left(Y_{T\times P}\right)\overrightarrow{w} = ||B_{K\times P}\overrightarrow{w}|| ^ 2$$ 
$s.t. \ \overrightarrow{w}\cdot\mathbb{1} = 1$ and $ \overrightarrow{w}^T\overrightarrow{\mu} \ge 0$ for long positions ($ \overrightarrow{w}^T\overrightarrow{\mu} \le 0$ for short positions).  
$\mu$ is the expected return of each security. 

## 4. Cluster Trading: Results and Discussion

### Assumptions

* Look-Back Period: 60 days (T).
* Systematic Principle Components: K = 3 for PCA. The first three components account for ~50% variance. 
* Signal Thresholding: Only when signal is within a importance level of 10% for either side. ($\Phi\left(s_{long,enter}\right)\approx 0.1, 1 - \Phi\left(s_{short,enter}\right)\approx 0.1$)
* Risk-Free (Bank) Returns: 5% annually equivalence. 

The strategy has been implemented with a set of securities. The authors want to make sure the PDF of the strategy return is right skewed.

### 4.1 Clustering Efficiency

Concatenated ETFs are the combination of all the ETF above.  
The clustering algo clusters the concatenated ETFs into 30 clusters, in general matching the breakdown of the ETFs.  
It detects that XLF (Finance) and XLFS (Finance) are similar, etc. 

![Table 1](Table1.png)

### 4.2 Trading Signal Statistics

Sanity Check to make sure the period taken is reasonable.  
Nyquist-Shannon sampling theorem.  
__If a function $x(t)$ contains no frequencies higher than B hertz, it is completely determined by giving its ordinates at a series of points spaced $1/(2B)$ seconds apart.__  
A sufficient sample-rate is therefore anything larger than 2B samples per second.  
https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem

In other words, when sampling from a periodical function, we need to make sure there are on average more than 2 samplings within one cycle.  
This is where I do not understand about the authors' point. They sampled the returns daily, so the shortest cycle should be 2 days. I don't know why they said 30 days.  
But from the plot we see the mean reversion time is in general larger than 2 days. So it is okay. 

![Figure 3](Figure3.png)

### 4.3 Trade Statistics

The distributions show a pattern of right skewed - thin negative tail and thick positive tail, 57% hit rate (0 is the 43rd percentile). 

To test the performance of the strategy regarding risk minimizing, the authors calculted the IQR (interquartile range, range between 3rd quartile and 2nd quartile).   
Given a dataset, for every trade completed, authors calculated the performance of a Markowitz portfolio containing the same assets as involved in the trade for the same holding period as the trade such that the expected return of the Markowitz portfolio is the attained return of the stat. arb. trade.  
The Markowitz is purely for weight assignment.  
__IQR and sharpe ratio of Markowitz do not outperform the strategy.__

The authors also try different signal thresholds for the strategy and got different hit rates. 

Baseline $s_{long,enter} = -1.25, s_{long,exit} = -0.5, s_{short,enter} = 1.25, s_{short,exit} = 0.75$  
Higher $s_{long,enter} = -1.5, s_{long,exit} = -0.75, s_{short,enter} = 1.5, s_{short,exit} = 1$  
Lower $s_{long,enter} = -1, s_{long,exit} = -0.25, s_{short,enter} = 1, s_{short,exit} = 0.5$

![Table 4](Table4.png)

## 5 Strategy 2: Combinatorial Pairs Trading at Different Frequencies

Assumptions:  
* This strategy is built on a generalization of the single-pair framework.  
* Among N assets, the authors consider each pair. And for each single time-step, single factor regression and AR(1）model parameters are fit for each pair. Then ranking the pairs according to F-score (expected number of reversions) and the correlation coefficient from the OU model, as well as the drift of the residual process. 
* Cleaning F-scores. Setting non-qualified F-scores as 0. 
* Allocate the capital according to signal strength. 

### 5.1 Simulation

The authors have two backtesting frameworks: 
* Low frequency (daily). No need to consider the limit order book. Data is from Yahoo Finance. 
* High frequency pricing data (microseconds). It contains the limit order data and also market liquidity. It is from Thesys Technologies. 

### 5.2 Trading at a Daily Frequency

The authors compare the performances of two portfolios with this strategy, the period is from July 2012 to May 2016: 
* Case 1: A portfolio with securities identified with the subtractive clustering algo from SPY and IWM. 
* Case 2: A portfolio containing selected large cap names. 

We can see the comparison between the two 
* Case 1 identifies the mispricing, with more trades, larger return and larger volatility.  
* Case 2 contains names not quite correlated, we see the returns are having steps due to low amount of trades. lower return and lower volatility. 


![Figure 6](Figure6.png)
![Table 5](Table5.png)

### 5.3 High Frequency

By increasing the frequency, the authors see __even more occurance of profitable signals__, but it may not be easily obtained.  
* The return of HFT is on the same level as the spread and the spread associated cost is not negligible. 
* The minimum tick size of 1 cent is too large to capture the signal (e.g. tick size is the minimum change in price allowed to be shown). 
* Transaction cost

![Figure 7](Figure7.png)

#### 5.3.1 Trading rules

Here are some rules to adjust the strategy to HFT idiosyncrasies.  
1. Stop-loss criterion: the authors block both assets in any pair if either one in the pair is current held, unless it is an exit signal. This is to minimize the transaction cost and limit the position held for each name. 
2. Liquidity Constraint: The authors set a coefficient __alpha__ to show the deviation of the execution price from the mid (alpha = 0 means trading at mid and alpha = 1 means taking the full BAS). The alpha is selected within 0.2 and 0.5 to represent the market. 
3. Unfilled order managementr: It is common to see order unfilled. For those unfilled ones, authors take actions like : 
    * if signal is still strong within the following time-step, recalibrate the model, compute the desired proportion and sumbit an order to rebalance the positions
    * if the signal is not strong anymore, submit orders to cancel the positon. 

Figure 8 shows the distribution of annualized returns of the strategy with a data frequency of 3 hrs.   
We see a fatter positive tail and a hit rate > 50%. 

![Figure 8](Figure8.png)

## 6. Results for pair strategy

Assess performance of pairs strategy in HFT.  
The pair of JPM and XLF is selected. 

### 6.1 Transaction costs vs gains

The authors implement the strategy on two frequencies: 3 hrs and 1 min. 
3-hr one has a range of 6 months and the other one has a range of 1 months while as expected the HFT one has many more trades within a shorted time range.  
The large amount of trades of HFT results in bad profitability: 
1. more trades results in larger transaction cost; 
2. high frequent pricing data drives the profit on similar level as the bid ask spread and it will be very sensitive to the timing. 

While bearing in mind that no conclusion is valid for all possible trading seasons
and regimes, this illustrates how trading costs can scale to undermine a potentially alarming proportion of gains.

### 6.2 Performance and risk measures

The authors implement the strategy on different frequencies for the same random selected starting date in H2 2015.  
In general, we see as the frequency goes up, the return goes down, which illustrates either: 
* signals generated at higher frequencies are spureious
* transaction costs overwhelm expected gain on each trade


![Table 6](Table6.png)

The observation shows that HFT contain strong non-continuous behavior, microstructure noise and jomps that undermine the usual estimator of the volatility.  
* Return decreases a lot as the frequency goes up
* Price vol is halved from 3 hrs to 10 sds. 
* Maximum drawdown increases to 4 times from 3 hrs to 10 sds. 
* Absolute VaR increases a lot, corresponding to the assumption that more frequent tradings bring up higher risk. 

To overcome the unrepresentativeness problem of the same starting date method above, the authors did another set of simulations:
* 30 independent simulations
* 1 min frequency
* the start dates are selected randomly between 2011 and 2015
The cumulative return of the 30 simulations are kde-ed into a pdf as Figure 11.

The graph is not quite right-skewed and the hit rate does not seem to be > 50%. 

![Figure 11](Figure11.png)

In HFT, we see non-continuous issue for the estimation of beta. And we can see in Figure 12 that there are jumps. 

![Figure 12](Figure12.png)

The authors also show the distributions of annualized returns across different frequencies. We can see as the frequency goes down, the pdf is flatter and more right skewed.  
This does not mean the pair trading strategy does not work in the HFT but __we need to bear in mind the adjustments to the strategy if we want to implement it well in HFT__. 

![Figure 13](Figure13.png)

## 7. Conclusion

### Summary - Cluster Trading

* The assumption is the mean reverting of the return on resisual of hedged similar assets. 
* PCA to decrease the amount of bases to K, each asset is a linear combination of the bases. 
* Use subtractive clustering to cluster the assets. 
* Allocate the weight to optimize the risk exposure. 
* Outperforms naive Markowitz portfolio. 

### Summary - High-Frequency Adaptation

It is not easy to profit in HFT. 
* Expected returns are at similar level as BAS. 
* Minimum tick size is larger to well utilize the signal. 
* Transaction costs
* Liquidity issue - orders may not be filled. 
The higher the frequency is, the more we need to consider the market capacity and other players. 

### Combination of Strategies

### Future Work

Below are some dimensions may potientially help the strategy
* Incorporation of the volume in the order book
* Have a dynamic alpha (deviation parameter of the execution price from the mid)
* Set dynamic thresholds for signals.
* Involve thresholding to neglect jumps in the OU process

# Summaries

* This paper is a course paper based on the statistical strategy and high frequency trading. 
* There are some typos and unclearness in the paper (explanation of some tables), but the logic is clear. 
* The authors discussed about two strategies: 1. a cluster of assets, 2. a set of pairs. 
* The authors show that strategy 1 has a hit rate > 50% and is profitable
* The authors discuss the performance of strategy 2 across different frequency of data and find out the current methodology cannot make profit in HFT though theoretically the arbitrage opportunity is no less in HFT. 
* The authors indicate that there are some paths may lead to better performance. 

I think we may consider these regarding the connection to ML: 
1. clustering method
2. use ML to learn dynamic alpha
3. use ML to learn dynamic threshold for signals